<a href="https://colab.research.google.com/github/gopalm-ai/Quantum_Machine_Learning_An_Applied_Approach/blob/main/TeleportationCircuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Libraries
! pip3 install cirq

In [2]:
import cirq
import random
import numpy as np
# Define three qubits: msg = qubit[0], qalice = qubit[1], qbob = qubit[2]
qubit=[0]*(3)
qubit[0] = cirq.NamedQubit('msg')
qubit[1] = cirq.NamedQubit('qalice')
qubit[2] = cirq.NamedQubit('qbob')
circuit = cirq.Circuit()
# Create a Bell state entangled pair to be shared between Alice and Bob.
circuit.append([cirq.H(qubit[1]), cirq.CNOT(qubit[1], qubit[2])])
# Creates a random state for the message
ranX = random.random()
ranY = random.random()
circuit.append([cirq.X(qubit[0])**ranX, cirq.Y(qubit[0])**ranY])

In [3]:
# Unitary operator rotating the two-qubit basis of the message and Alice's 
# entangled qubit rotates the Bell state basis to the computational basis:
circuit.append([cirq.CNOT(qubit[0], qubit[1]), cirq.H(qubit[0])])
# combining now with a measurement in the computational basis,
# we effectively have projected this two-qubit state onto one of the four 
# states of the Bell state basis:
circuit.append(cirq.measure(qubit[0], qubit[1]))

In [4]:
# Use the two classical bits from the Bell measurement to recover the original 
# message on Bob's entangled qubit.
circuit.append([cirq.CNOT(qubit[1], qubit[2]), cirq.CZ(qubit[0], qubit[2])])
print("Circuit:")
print(circuit)

Circuit:
msg: ──────X^0.137───Y^0.046───@───H───M───────@───
                               │       │       │
qalice: ───H─────────@─────────X───────M───@───┼───
                     │                     │   │
qbob: ───────────────X─────────────────────X───@───


In [5]:
sim = cirq.Simulator()
# Simulation that applies the random X and Y gates to
# create the message.
q0 = cirq.LineQubit(0)
message = sim.simulate(cirq.Circuit([cirq.X(q0)**ranX, cirq.Y(q0)**ranY]))
print("\nBloch sphere of the message qubit in the initial state:")
expected = cirq.bloch_vector_from_state_vector(message.final_state_vector,0)
print("x: ", np.around(expected[0], 4), " y: ", np.around(expected[1], 4), 
      " z: ", np.around(expected[2], 4))


Bloch sphere of the message qubit in the initial state:
x:  0.1308  y:  -0.4165  z:  0.8997


In [6]:
# Records the final state of the simulation.
final_results = sim.simulate(circuit)
print("\nBloch sphere of Bob's qubit in the final state:")
teleported = cirq.bloch_vector_from_state_vector(final_results.final_state_vector, 2)
print("x: ", np.around(teleported[0], 4), " y: ", np.around(teleported[1], 4), 
      " z: ", np.around(teleported[2], 4))
print("\nBloch sphere of message qubit in the final state:")
message_final = cirq.bloch_vector_from_state_vector(final_results.final_state_vector, 0)
print("x: ", np.around(message_final[0], 4), " y: ", np.around(message_final[1], 4), 
      " z: ", np.around(message_final[2], 4))


Bloch sphere of Bob's qubit in the final state:
x:  0.1308  y:  -0.4165  z:  0.8997

Bloch sphere of message qubit in the final state:
x:  0.0  y:  0.0  z:  1.0


In [7]:
# Unitary operator rotating the two-qubit basis of the message and Alice's 
# entangled qubit; rotates Bell state basis to the computational basis:
circuit.append([cirq.CNOT(qubit[0], qubit[1]), cirq.H(qubit[0])])
# This time skip the measurement
# circuit.append(cirq.measure(qubit[0], qubit[1]))
# Use the same operations as before to recover the original quantum message on
# Bob's entangled qubit
circuit.append([cirq.CNOT(qubit[1], qubit[2]), cirq.CZ(qubit[0], qubit[2])])